<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Template para o Colab do Projeto Semestral**
---

Atenção, podem ser que nem todas as tarefas sejam executadas no Colab (a aplicação por exemplo, pode estar hospedada no streamlit cloud). Mas a maior parte pode estar aqui ou ao menos indicada e comentada.


Além disso a entrega deve incluir:

1. **Um GitHub público do projeto**
2. **Código completo e executável em um notebook Python (este template)**
3. **Uma aplicação streamlit para consumo do modelo**
4. **Um texto/artigo do projeto**
5. **Um vídeo (link YouTube ou outro) de no máximo 3min de apresentação do projeto**

Um **`readme.md`** no GitHub público do projeto deve indicar (um índice) cada uma dessas entregas.








In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética
Aluno1 = '10403109, Erik Samuel Viana Hsu' #@param {type:"string"}
Aluno2 = '10400995, Mateus Kenzo Iochimoto' #@param {type:"string"}
Aluno3 = '10400764, Thiago Shihan Cardoso Toma' #@param {type:"string"}



In [ ]:
#@title Assinale aqui a sua opção de Projeto
Projeto = "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos" #@param ["IA Aplicada a Imagens: Uso de Modelos de Redes Neurais", "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos"]




# **Resumo**

# **1. Objetivo do projeto**

O objetivo deste projeto é construir um sistema de **tradução automática neural** do inglês para o português, utilizando modelos pré-treinados da biblioteca **Hugging Face Transformers**. A proposta é aplicar técnicas modernas de NLP para adaptar um modelo de tradução com dados personalizados, visando melhorar sua performance em um domínio específico.

# **2. Fontes dos dados e dados originais (coletados)**
Os dados utilizados neste projeto foram extraídos do **EMEA Parallel Corpus** (European Medicines Agency), um conjunto de frases paralelas em inglês e português, focado no domínio biomédico. Os arquivos utilizados foram:

- `EMEA.en-pt.en` — Frases em inglês  
- `EMEA.en-pt.pt` — Traduções correspondentes em português

# 3. **Ferramentas/pacotes de IA a serem utilizados para a construção da solução**

- **Transformers** — Para carregamento e fine-tuning do modelo `Helsinki-NLP/opus-mt-en-ROMANCE`
- **Datasets** — Para manipulação e tokenização dos dados
- **Evaluate** — Para cálculo da métrica `sacreBLEU`
- **PyTorch** — Backend para o treinamento

# 4. **Uma prévia dos resultados.**
O modelo foi ajustado com 2000 pares de frases para treino e 500 para avaliação.

Para testar a função de tradução, passamos uma frase em inglês como parâmetro de entrada e a saída é a frase traduzida em português. Por exemplo, se a frase for "Keep medicine away from children", a tradução será "Manter os medicamentos longe das crianças". Outras frases e entradas podem ser testadas.


# **Apresentação dos dados**


Utilizaremos o corpus EMEA (European Medicines Agency), que contém traduções de bulas de medicamentos em vários idiomas. Os dados estão disponíveis publicamente via [OPUS](https://opus.nlpl.eu/EMEA.php).

In [1]:
!wget https://object.pouta.csc.fi/OPUS-EMEA/v3/moses/en-pt.txt.zip -O emea_en-pt.zip
!unzip emea_en-pt.zip

--2025-05-31 13:48:05--  https://object.pouta.csc.fi/OPUS-EMEA/v3/moses/en-pt.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35242984 (34M) [application/zip]
Saving to: ‘emea_en-pt.zip’

emea_en-pt.zip      100%[===================>]  33.61M  11.1MB/s    in 3.0s    

2025-05-31 13:48:09 (11.1 MB/s) - ‘emea_en-pt.zip’ saved [35242984/35242984]

Archive:  emea_en-pt.zip
  inflating: EMEA.en-pt.en           
  inflating: EMEA.en-pt.pt           
  inflating: README                  


In [2]:
# Visualização de algumas amostras
with open("EMEA.en-pt.en", encoding="utf-8") as f_en, open("EMEA.en-pt.pt", encoding="utf-8") as f_pt:
    for _ in range(5):
        print("EN:", f_en.readline().strip())
        print("PT:", f_pt.readline().strip())
        print("---")

EN: European Medicines Agency
PT: European Medicines Agency
---
EN: EMEA/ H/ C/ 471
PT: EMEA/ H/ C/ 471
---
EN: EUROPEAN PUBLIC ASSESSMENT REPORT (EPAR)
PT: RELATÓRIO PÚBLICO EUROPEU DE AVALIAÇÃO (EPAR)
---
EN: ABILIFY
PT: ABILIFY
---
EN: EPAR summary for the public
PT: Resumo do EPAR destinado ao público
---


# **Preparação e transformação dos dados**



In [3]:
!pip install --upgrade transformers
!pip install evaluate sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.1 MB/s eta 0:00:00


In [4]:
from datasets import Dataset

# Função que carrega os dados personalizados de tradução a partir de 2 arquivos de texto
def load_custom_data():
    with open("EMEA.en-pt.en", encoding="utf-8") as f_en, open("EMEA.en-pt.pt", encoding="utf-8") as f_pt:
        data = [{"translation": {"en": en.strip(), "pt": pt.strip()}} for en, pt in zip(f_en, f_pt)] # cria uma lista de dicionários, onde cada item é um par de tradução
    return Dataset.from_list(data) # converte a lista de pares de tradução em um objeto Dataset do Hugging Face

dataset = load_custom_data() # carrega o dataset personalizado com a função acima
dataset = dataset.train_test_split(test_size=0.1) # divide o dataset em dois subconjuntos -> teste (10%) e treino (90%)
dataset # exibe o dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 973929
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 108215
    })
})

# **Fine Tuning do modelo**


Nessa etapa, utilizamos o modelo `Helsinki-NLP/opus-mt-en-ROMANCE` (o -pt era público mas ficou privado, mas o ROMANCE contempla pt também) com ajustes finos baseados no nosso conjunto técnico (bula de remédios).

In [5]:
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# Define o nome do modelo base para tradução do inglês para as línguas românicas (inclui pt, es, fr, etc)
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"

#Carrega o tokenizer, transformando os textos em tokens
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Carrega o modelo pré treinado de tradução com base no nome fornecido
model = MarianMTModel.from_pretrained(model_name)

#Função que pré-processa os dados de entrada para o treinamento
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]] # extrai os textos em inglês da estrutura translation do dataset
    targets = [ex["pt"] for ex in examples["translation"]] # extrai as traduções para o português
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length") # tokeniza os textos de entrada
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length") #tokeniza os textos-alvo
    model_inputs["labels"] = labels["input_ids"] # add os labels tokenizados ao dicionário de entrada
    return model_inputs

# Aplica a função de pré processamento a todo o dataset, de forma batched
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/779k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/799k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

Map:   0%|          | 0/973929 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/108215 [00:00<?, ? examples/s]

In [6]:
# Definindo os argumentos / hiperparâmetros de treinamento
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
)

small_train_dataset = tokenized_dataset["train"].select(range(2000)) # dataset de treino
small_eval_dataset = tokenized_dataset["test"].select(range(500)) #dataset de teste

# Trainer com modelo de tradução Seq2Seq
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

trainer.train() # inicia o treinamento

<ipython-input-6-62e1620b5e6c>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erik0608br (erik0608br-mackenzie) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.138679
2,0.198800,0.137725


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.1988022003173828, metrics={'train_runtime': 5833.4726, 'train_samples_per_second': 0.686, 'train_steps_per_second': 0.086, 'total_flos': 135593459712000.0, 'train_loss': 0.1988022003173828, 'epoch': 2.0})

# **Avaliação do modelo**



Para avaliar a qualidade de tradução, usamos a métrica BLEU (sacrebleu).

In [7]:
import evaluate

metric = evaluate.load("sacrebleu") # carrega metrica sacreBLEU

def compute_metrics(eval_preds):
    # Descompacta os pares de (predições, rótulos/labels)
    preds, labels = eval_preds

    # Decodifica as predições
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Substitui -100 pelo token de pad no rótulo
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]

    # Decodifica os rótulos reais (targets) para texto legível, ignorando tokens especiais
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # SacreBLEU espera lista de listas no campo `references`
    return metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])

# **Consumo do modelo**

Função para traduzir frases novas com o modelo fine-tuned

In [10]:
def translate(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True) #tokeniza o texto de entrada
    translated = model.generate(**inputs) # gera tradução usando o modelo carregado
    return tokenizer.decode(translated[0], skip_special_tokens=True) # decodifica a saída gerada para texto legível

# Exemplo de uso
exemplo = "Keep medicine away from children."
print("Tradução:", translate(exemplo))

Tradução: Manter os medicamentos longe das crianças.


# **Referências**

- OPUS EMEA Corpus: https://opus.nlpl.eu/EMEA.php  
- MarianMT - Hugging Face: https://huggingface.co/Helsinki-NLP/opus-mt-en-pt  
- Hugging Face Transformers: https://huggingface.co/docs/transformers/index  
- Streamlit: https://streamlit.io/

---

In [ ]:
#@title **Avaliação**
GitHub = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao_Model_Code = 7 #@param {type:"slider", min:0, max:10, step:1}

Aplicacao_Streamlit = 9 #@param {type:"slider", min:0, max:10, step:1}

Texto_Artigo  = 6 #@param {type:"slider", min:0, max:10, step:1}

Video = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**

nota = 2*GitHub + 4*Implementacao_Model_Code + 2*Aplicacao_Streamlit + 1*Texto_Artigo + 1*Video

nota = nota / 10

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.9



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.9
1,1115677,DANIEL HENRIQUE,7.9
